In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')

jobclass1 = ClosedClass(model, 'Class1', 10, P1, 0)
jobclass2 = ClosedClass(model, 'Class2', 7, P1, 0)

In [3]:
# T1 - Different modes for different job classes
# Mode1: handles Class1 - requires 2 tokens
mode1 = T1.add_mode('Mode1')
T1.setDistribution(mode1, Exp(2))
T1.setEnablingConditions(mode1, jobclass1, P1, 2)
T1.setFiringOutcome(mode1, jobclass1, P2, 2)

# Mode2: handles Class2 - requires 1 token
mode2 = T1.add_mode('Mode2')
T1.setDistribution(mode2, Exp(3))
T1.setEnablingConditions(mode2, jobclass2, P1, 1)
T1.setFiringOutcome(mode2, jobclass2, P2, 1)

# T2 - Handles Class1 tokens individually
mode3 = T2.add_mode('Mode3')
T2.setDistribution(mode3, Erlang(1.5, 2))
T2.setEnablingConditions(mode3, jobclass1, P2, 1)
T2.setFiringOutcome(mode3, jobclass1, P1, 1)

# T3 - Handles Class2 tokens in batches of 4
mode4 = T3.add_mode('Mode4')
T3.setDistribution(mode4, Exp(0.5))
T3.setEnablingConditions(mode4, jobclass2, P2, 4)
T3.setFiringOutcome(mode4, jobclass2, P1, 4)

In [4]:
# Set up routing matrix for both job classes
routingMatrix = model.init_routing_matrix()

# Class1 routing
routingMatrix.set(jobclass1, jobclass1, P1, T1, 1.0)  # P1 -> T1 (Class1)
routingMatrix.set(jobclass1, jobclass1, P2, T2, 1.0)  # P2 -> T2 (Class1)
routingMatrix.set(jobclass1, jobclass1, T1, P2, 1.0)  # T1 -> P2 (Class1)
routingMatrix.set(jobclass1, jobclass1, T2, P1, 1.0)  # T2 -> P1 (Class1)

# Class2 routing
routingMatrix.set(jobclass2, jobclass2, P1, T1, 1.0)  # P1 -> T1 (Class2)
routingMatrix.set(jobclass2, jobclass2, P2, T3, 1.0)  # P2 -> T3 (Class2)
routingMatrix.set(jobclass2, jobclass2, T1, P2, 1.0)  # T1 -> P2 (Class2)
routingMatrix.set(jobclass2, jobclass2, T3, P1, 1.0)  # T3 -> P1 (Class2)

model.link(routingMatrix)

In [5]:
# Set initial state - both classes start in P1
# P1 gets [10, 7] tokens for [Class1, Class2]
P1.setState([10, 7])  # Using Python list for multi-class
P2.setState([0, 0])  # No tokens in P2 for either class

In [6]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/3023745069890190980/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.595659s.
  Station JobClass    QLen  Util    RespT   ResidT    ArvR    Tput
0      P1   Class1  0.8811   0.0   1.1688   1.1688  0.7533  0.7549
1      P1   Class2  1.7080   0.0   1.1162   1.1162  1.5906  1.6042
2      P2   Class1  9.1189   0.0  12.1005  12.1005  0.7549  0.7550
3      P2   Class2  5.2920   0.0   3.3423   3.3423  1.6042  1.5821
